In [12]:
from xml.etree import ElementTree
tree = ElementTree.parse('/home/2577B55/phddata/customer_profile_data-1556930512566.xml')
root = tree.getroot()
f= open('/home/2577B55/phddata/customer_profile_data-new.csv',"w+")

for customer_profile in root:
    customerID= customer_profile.find('customerID').text
    gender= customer_profile.find('gender').text
    SeniorCitizen= customer_profile.find('SeniorCitizen').text
    Partner= customer_profile.find('Partner').text
    Dependents= customer_profile.find('Dependents').text
    tenure= customer_profile.find('tenure').text
    PhoneService= customer_profile.find('PhoneService').text
    MultipleLines= customer_profile.find('MultipleLines').text
    InternetService= customer_profile.find('InternetService').text
    OnlineSecurity= customer_profile.find('OnlineSecurity').text
    OnlineBackup= customer_profile.find('OnlineBackup').text
    DeviceProtection= customer_profile.find('DeviceProtection').text
    TechSupport= customer_profile.find('TechSupport').text
    StreamingTV= customer_profile.find('StreamingTV').text
    StreamingMovies= customer_profile.find('StreamingMovies').text
    Contract= customer_profile.find('Contract').text
    PaperlessBilling= customer_profile.find('PaperlessBilling').text
    Churn= customer_profile.find('Churn').text
    f.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,Churn))

In [ ]:
kafka-topics.sh --create --zookeeper c.insofe.edu.in:2181 --replication-factor 1 --partitions 1 -- topics 2577B55_phd_topic

In [ ]:
kafka-topics.sh --list --zookeeper c.insofe.edu.in:2181 |  grep '2577B55_phd_topic'

In [ ]:
agent.sources = Flumesrc
agent.channels = memoryChannel
agent.sinks = flumeKafka

# Setting the source to spool directory where the file exists
agent.sources.Flumesrc.type = spooldir
agent.sources.Flumesrc.spoolDir = /home/2577B55/phddata    

# Setting the channel to memory
agent.channels.memoryChannel.type = memory
# Max number of events stored in the memory channel
agent.channels.memoryChannel.capacity = 10000
agent.channels.memoryChannel.transactioncapacity = 1000

# Setting the sink to Kafka
agent.sinks.flumeKafka.type = org.apache.flume.sink.kafka.KafkaSink
agent.sinks.flumeKafka.topic = 2577B55_phd_topic
agent.sinks.flumeKafka.brokerList = c.insofe.edu.in:9092
agent.sinks.flumeKafka.batchSize = 20


# Connect source and sink with channel
agent.sources.Flumesrc.channels = memoryChannel
agent.sinks.flumeKafka.channel = memoryChannel


In [2]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.6-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [4]:
from pyspark.sql import SparkSession

# Import the necessary classes and create a local SparkSession, the starting point of all functionalities related to Spark.
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col,udf

from pyspark.ml import Pipeline, PipelineModel

spark = SparkSession \
    .builder \
    .appName("Kafka Spark Structured Streaming") \
    .config("spark.master", "local") \
    .getOrCreate()




In [5]:
spark.sparkContext.setLogLevel("ERROR")


# Reading the messsages from the kafka topic and creating a data frame
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "c.insofe.edu.in:2181") \
  .option("subscribe", "2577B55_phd_topic") \
  .option("startingOffsets", "earliest") \
  .load()



Py4JJavaError: An error occurred while calling o73.load.
: java.lang.ClassNotFoundException: Failed to find data source: kafka. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:635)
	at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:159)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.Gateway

In [ ]:
df.write.csv("hdfs://user/2577B55/customer_profile.csv")

In [ ]:
df.writeStream.format("parquet").outputMode("append").option("truncate","false").option("path", "hdfs://user/2577B55/customer_profile.csv").option("checkpointLocation", "file:///home/2567B55/Uber_Use_Case/results/outputCP").start()

In [ ]:
#Run this command from conf folder
flume-ng agent --conf-file flume.properties -Dflume.root.logger=DEBUG,console --name agent

In [ ]:
#Run the python code from folder priya-python code
spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0 kafka-to-hdfs.py